In [ ]:
# 1
# --- CÀI ĐẶT THƯ VIỆN & TẢI DỮ LIỆU ---
!pip install roboflow ultralytics -q
from roboflow import Roboflow
from ultralytics import YOLO
import yaml

# --- KHU VỰC 1: NGUỒN 1 (Đã sửa biến thành dataset1) ---
rf = Roboflow(api_key="KbCbXZFXVVlWPQLUljGk")
project = rf.workspace("seprinder").project("detect-failure-in-3d-printing")
version = project.version(10)
dataset1 = version.download("yolov8")  # <--- Đã sửa thành dataset1

# --- KHU VỰC 2: NGUỒN 2 (Đã sửa biến thành dataset2) ---
rf = Roboflow(api_key="KnQeHpU9nBvR4DJKZAZN")
project = rf.workspace("seprinder-2").project("detect-failure-in-3d-printing-2")
version = project.version(4)
dataset2 = version.download("yolov8")  # <--- Đã sửa thành dataset2


# ==============================================================================
# PHẦN XỬ LÝ TỰ ĐỘNG & HUẤN LUYỆN
# ==============================================================================
print(f"\n✅ Đã tải xong: \n1. {dataset1.location}\n2. {dataset2.location}")

# 1. Đọc thông tin class từ dataset 1
with open(f'{dataset1.location}/data.yaml', 'r') as f:
    base_config = yaml.safe_load(f)

# 2. Tạo file cấu hình gộp (merged.yaml)
merged_config = {
    'train': [f'{dataset1.location}/train/images', f'{dataset2.location}/train/images'],
    'val':   [f'{dataset1.location}/valid/images', f'{dataset2.location}/valid/images'],
    'names': base_config['names'],
    'nc': base_config['nc']
}

with open('merged.yaml', 'w') as f:
    yaml.dump(merged_config, f)

print("✅ Đã tạo file cấu hình gộp: merged.yaml")

# 3. Tiến hành Huấn luyện (Train) ngay lập tức
print("🚀 Bắt đầu huấn luyện...")
model = YOLO('yolov8n.pt')

results = model.train(
    data='merged.yaml',   # Trỏ vào file gộp
    epochs=50,
    imgsz=512,
    batch=-1,
    plots=True
)

In [ ]:
# 2
# Hiển thị kết quả huấn luyện.
import glob
from IPython.display import Image, display

# Tìm file kết quả (lấy file png đầu tiên tìm thấy trong thư mục train mới nhất)
try:
    results_path = glob.glob('/content/runs/detect/train*/results.png')[0]
    display(Image(filename=results_path))
    print(f"Đang hiển thị kết quả từ: {results_path}")
except IndexError:
    print("Chưa tìm thấy ảnh kết quả. Có thể quá trình train chưa xong hoặc bị lỗi.")

In [ ]:
# 3
# Load kết quả huấn luyện.
# 1. Load mô hình vừa train xong
model = YOLO('/content/runs/detect/train/weights/best.pt')  # Chỉnh lại đường dẫn train/train2 cho đúng thư mục của bạn

import glob

# Tìm một hình ảnh hợp lệ từ thư mục 'valid/images' của dataset1
# Đảm bảo đường dẫn này tồn tại sau khi dataset được tải xuống
image_path = glob.glob(f'{dataset1.location}/valid/images/*.jpg')[0]
print(f"Đang sử dụng hình ảnh cho dự đoán: {image_path}")

# 2. Chạy predict với ngưỡng cực thấp để "ép" nó phải nói ra
results = model.predict(
    source='/content/Spagettimon.jpg',
    save=True,
    conf=0.1 # Chấp nhận độ tin cậy thấp 10%
)

# 3. Xem kết quả
for r in results:
    print(f"Phát hiện: {r.boxes.conf}") # In ra xem nó tự tin bao nhiêu %

In [ ]:
# 4
# Hiển thị kết quả dự đoán.
# Lấy ảnh vừa vẽ khung xong hiện ra màn hình
Image(filename=glob.glob('runs/detect/predict/*.jpg')[0])

In [ ]:

# 5
# 1. Load lại model tốt nhất vừa train
model = YOLO('/content/runs/detect/train/weights/best.pt')

# 2. Xuất sang định dạng ONNX (Tối ưu cho chạy trên Client/CPU)
# imgsz=512 phải khớp với lúc bạn train
# Cần cố định opset = 18 để có thể chạy được trên raspberry pi 3/4.
path = model.export(format='onnx', imgsz=512, opset=18)

print(f"File ONNX đã được lưu tại: {path}")